In [166]:
%%bash
for i in 1 2 3 4 5 6 7 8 9;
do
    zcat ~/data/encode/hg19/E00${i}-H3K27me3.fc.signal.bigwig.genome.sorted.gz | head -n 1000000 | gzip > /tmp/bgcomp${i}
done;

In [167]:
import gzip
import numpy as np
import os
import os.path as op
import time as time

# 
#filepath = '/data/encode/hg19/E116-DNase.fc.signal.bigwig.bedGraph.genome.sorted.short.gz'
filepath='/tmp/bgcomp1'

In [168]:
t1 = time.time()
uniq_values = set()

with gzip.open(filepath, 'r') as f:
    for n, line in enumerate(f):
        parts = list(map(float, line.split()))
        uniq_values.add(parts[2])

print("time to read:", time.time() - t1)
print("unique_values:", len(uniq_values))

time to read: 2.2976067066192627
unique_values: 9905


In [172]:
import csv

t1 = time.time()

buffers = []

for i in [1,2,3,4,5,6,7,8,9]:
    filepath='/tmp/bgcomp{}'.format(i)
    with gzip.open(filepath, 'rt') as f:
        buffer = []
        prev_value = 0

        for n,line in enumerate(csv.reader(f, delimiter='\t')):
            parts = line

            if parts[0] != 0:
                buffer += [float(parts[2]) for i in range(int(parts[0]), int(parts[1]))]
    buffers += [np.array(buffer)]
min_length = min([len(b) for b in buffers])
buffers = [b[:min_length] for b in buffers]
buffers = [b / sum(b) for b in buffers]


In [173]:
indeces = np.random.choice(range(min_length), 10000)
sub_buffers = [b[indeces] for b in buffers]

In [174]:
from sklearn.decomposition import PCA
pca = PCA(n_components=1)
one_d_colours = pca.fit_transform(sub_buffers)[:,0]
print("one_d_colours", one_d_colours)
ix = np.argsort(one_d_colours)
print("ix:", ix + 1)


one_d_colours [  9.74869861e-06   2.20621840e-07   3.69230656e-06  -6.47907681e-07
  -2.54037642e-06  -3.65589585e-06  -3.34799457e-06  -2.43895346e-06
  -1.03049904e-06]
ix: [6 7 5 8 9 4 2 3 1]
